In [ ]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')
    
from DSProject import DSProject
import pandas as pd
from model import *

dsp = DSProject()

In [ ]:
train = dsp.read_table(train_dataset + '_processed', 'feature')

In [ ]:
validate = dsp.read_table(validate_dataset + '_processed', 'feature')

In [ ]:
score = dsp.read_table(score_dataset + '_processed', 'feature')

Feature selection

In [ ]:
x, y, col_to_drop_numeric, col_to_drop_cat = feature_selection(train, 'group1')

In [ ]:
x_validate, y_validate, _, _ = feature_selection(validate, 'group1', col_to_drop_numeric, col_to_drop_cat)

In [ ]:
x_score, y_score, _, _ = feature_selection(score, None, col_to_drop_numeric, col_to_drop_cat)

Modeling

In [ ]:
mod, result, (x_train, x_test, y_train, y_test) = train_classifier(x, y)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

def adjusted_threshold_performance(mod, x_test, y_test, df, threshold=0.7):
    p = mod.predict_proba(x_test)
    y_true = np.array(y_test.tolist(), dtype=int)
    y_pred = np.array(p[:, 1] > threshold, dtype=int)
    result = classification_report(y_true, y_pred)
    print('Group 1 prediction performance after adjusting threshold')
    print(result)

    # self-cure classification metrics
    confm = confusion_matrix(y_true, y_pred)
    total_test_pop = y_test.shape[0]
    error = confm[0, 1] * 100 / total_test_pop
    print("Self-cure confusion matrix")
    confusion_table = pd.DataFrame(confm, columns=['Predict 0', 'Predict 1'], index=['Label 0', 'Label 1'])
    print(confusion_table)

    print("Number of people tested")
    print(total_test_pop)
    print("Number of people predicted to be class 1")
    print(confm[0, 1] + confm[1, 1])
    print("Percent of people removed")
    print((confm[0, 1] + confm[1, 1]) * 100 / total_test_pop)
    print("Percentage of mislabeled people")
    print(error)

    test = y_test.to_frame()
    true_labels = df['Group'].iloc[test.loc[(y_true == 0) & (y_pred == 1)].index]
    print("Percent of action-roll people who got mistakenly eliminated in this step:")
    print(true_labels.value_counts()[5] * 100 / total_test_pop)

In [ ]:
print("Adjusted-threshold Group1 Results (Testing Set)")
adjusted_threshold_performance(mod['RFC'], x_test, y_test, train)

In [ ]:
print("Adjusted-threshold Group1 Results (Validation Set)")
adjusted_threshold_performance(mod['RFC'], x_validate, y_validate, validate)

In [ ]:
def adjusted_threshold_prediction(mod, x, y, df, threshold=0.7):
    p = mod.predict_proba(x)
    df['group1_p'] = np.array(p[:, 1])
    df['group1_prediction'] = np.array(p[:, 1] > threshold, dtype=int)
    nongroup1 = df.loc[df['group1_prediction'] == 0]
    return nongroup1, df

In [ ]:
nongroup1, train = adjusted_threshold_prediction(mod['RFC'], x, y, train)

In [ ]:
dsp.write_table(train, train_dataset + '_group1_prediction', 'output')

In [ ]:
nongroup1, validate = adjusted_threshold_prediction(mod['RFC'], x_validate, y_validate, validate)

In [ ]:
dsp.write_table(validate, validate_dataset + '_group1_prediction', 'output')

In [ ]:
nongroup1, score = adjusted_threshold_prediction(mod['RFC'], x_score, y_score, score)

In [ ]:
dsp.write_table(score, score_dataset + '_group1_prediction', 'output')

Check group1 prediction

In [ ]:
score['group1_prediction'].value_counts()/len(score.index)

Feature importance

In [ ]:
make_feature_importance_table(mod['RFC'], x_train, 'feature_importance_group1')